養成每日寫data science code的習慣！
改用慢慢實作專案累積知識以及實作能力，以 CV、NLP、GAN為主！

## 目標: 增加更多 Prototype 以及各應用實作能力！

### 本次目標
> 提升 BERT 客製化能力，可以應用在更特定的場域上，比如 FinBERT、NewsBERT 等等。

### 下載需要的套件

In [1]:
!pip install git+https://github.com/huggingface/transformers.git
!pip install datasets
!pip install huggingface-hub
!pip install nltk

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/huggingface/transformers.git to c:\users\aband\appdata\local\temp\pip-req-build-c1yxoi5l
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for transformers: filename=transformers-4.26.0.dev0-py3-none-any.whl size=5858133 sha256=eeaabb30f1f9b53dc25089fb7e409dc969060c55fc289dd3fe1e496ec841d1a9
  Stored in directory: C:\Users\aband\AppData\Local\Temp\pip-ephem-wheel-cache-24_bw57i\wheels\90\a5\44\6bcd83827c8a60628c5ad602f429cd5076bcce5f2a90054947
Successfully built transformers
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.10.3
    Uninstalling tokenizers-0.10.3:
      Success

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
  Running command git clone -q https://github.com/huggingface/transformers.git 'C:\Users\aband\AppData\Local\Temp\pip-req-build-c1yxoi5l'
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 1.9.0 requires huggingface-hub<0.1.0, but you have huggingface-hub 0.11.1 which is incompatible.
You should consider upgrading via the 'c:\program files (x86)\python\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.11.1
    Uninstalling huggingface-hub-0.11.1:
      Successfully uninstalled huggingface-hub-0.11.1


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.26.0.dev0 requires huggingface-hub<1.0,>=0.10.0, but you have huggingface-hub 0.0.19 which is incompatible.
You should consider upgrading via the 'c:\program files (x86)\python\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
You should consider upgrading via the 'c:\program files (x86)\python\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
You should consider upgrading via the 'c:\program files (x86)\python\python.exe -m pip install --upgrade pip' command.


In [2]:
import nltk
import random
import logging

import tensorflow as tf
from tensorflow import keras

nltk.download("punkt")
# Only log error messages
tf.get_logger().setLevel(logging.ERROR)
# Set random seed
tf.keras.utils.set_random_seed(42)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aband\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


### 設定超參數、變數

In [3]:
TOKENIZER_BATCH_SIZE = 256
TOKENIZER_VOCAB =  25000

BLOCK_SIZE = 128 # 也可以說是 seq_len
NSP_PROB = 0.5 # NSP 任務上下是流貫的比例
SHORT_SEQ_PROB = 0.1 # 
MAX_LENGTH = 512

MLM_PROB = 0.2   # MLM 任務變成 [MASK] 的比例

TRAIN_BATCH_SIZE = 2   # 預訓練模型，因為預訓練運算記憶體所需很大，BATCH_SIZE大不了
MAX_EPOCHS = 1
LEARNING_RATE = 1e-4

MODEL_CHECKPOINT = 'bert-base-cased'

### 取得資料集

In [4]:
from datasets import load_dataset

dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

Downloading:   0%|          | 0.00/1.93k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.72M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset wikitext downloaded and prepared to C:\Users\aband\.cache\huggingface\datasets\wikitext\wikitext-2-raw-v1\1.0.0\aa5e094000ec7afeb74c3be92c88313cd6f132d564c7effd961c10fd47c76f20. Subsequent calls will reuse this data.


看看資料集

In [5]:
print(dataset)

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})


In [12]:
len(dataset['train']['text'])

36718

In [18]:
# 取出一筆來看，可以發現是一個個句子、小片段，也是為了符合模型輸出(最大限制512 tokens with padding)

dataset['train']['text'][5]

" It met with positive sales in Japan , and was praised by both Japanese and western critics . After release , it received downloadable content , along with an expanded edition in November of that year . It was also adapted into manga and an original video animation series . Due to low sales of Valkyria Chronicles II , Valkyria Chronicles III was not localized , but a fan translation compatible with the game 's expanded edition was released in 2014 . Media.Vision would return to the franchise with the development of Valkyria : Azure Revolution for the PlayStation 4 . \n"

### MLM 任務前處理

為了可以去訓練 MLM 任務，需要將 raw data 去做 masking。
但其實 transformers 有提供 MLM、NSP處理的套件。

In [19]:
from transformers import DataCollatorForLanguageModeling



ImportError: cannot import name 'CommitOperationAdd' from 'huggingface_hub' (C:\Users\aband\AppData\Roaming\Python\Python37\site-packages\huggingface_hub\__init__.py)